<a href="https://colab.research.google.com/github/ccastano1997/predictionV2/blob/main/55k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 6.8 MB/s eta 0:00:00


In [ ]:
# ==========================================
# 🏀 NBA PREDICTION MODEL V2.0 (Advanced)
# ==========================================
# Features: LSTM Neural Network, Four Factors Analytics, Live Odds, Injury Audit
# ==========================================

import pandas as pd
import numpy as np
import time
from nba_api.stats.endpoints import leaguegamefinder, leaguedashplayerstats
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras import regularizers

# --- CONFIGURATION ---
SEASON = '2025-26'
SEQ_LENGTH = 5  # Games to look back
ODDS_FILE = 'nba_odds_full.csv'

# ==========================================
# 1. LOAD AND PROCESS ODDS DATA
# ==========================================
print(f"--- STEP 1: Loading Betting Data ({ODDS_FILE}) ---")
try:
    odds_df = pd.read_csv(ODDS_FILE)
    odds_df['Date'] = pd.to_datetime(odds_df['Date'], format='%d %b %Y')
except FileNotFoundError:
    print(f"❌ Error: '{ODDS_FILE}' not found. Please upload it.")
    raise

# Convert American Odds to Implied Probability
def american_to_prob(odd_str):
    try:
        if pd.isna(odd_str) or odd_str == 'v': return 0.5
        odd = float(odd_str)
        if odd > 0: return 100 / (odd + 100)
        else: return abs(odd) / (abs(odd) + 100)
    except: return 0.5

odds_df['Home_Prob'] = odds_df['Home Odds'].apply(american_to_prob)
odds_df['Away_Prob'] = odds_df['Away Odds'].apply(american_to_prob)

# Split Training (Past Results) vs Prediction (Tonight)
train_odds = odds_df[odds_df['Score'] != 'v'].copy()
predict_odds = odds_df[odds_df['Score'] == 'v'].copy()
print(f"✅ Loaded {len(train_odds)} past games for training.")
print(f"✅ Found {len(predict_odds)} games to predict tonight.")

# ==========================================
# 2. FETCH NBA STATS & FEATURE ENGINEERING
# ==========================================
print(f"\n--- STEP 2: Fetching {SEASON} Game Stats ---")
gamefinder = leaguegamefinder.LeagueGameFinder(
    season_nullable=SEASON,
    league_id_nullable='00',
    season_type_nullable='Regular Season'
)
all_games = gamefinder.get_data_frames()[0]
all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE'])
all_games = all_games.sort_values('GAME_DATE')
all_games = all_games[all_games['WL'].notna()] # Remove games not played yet

# --- CALCULATE "FOUR FACTORS" (Advanced Stats) ---
# 1. Effective Field Goal % (Adjusts for 3-pointers)
all_games['EFG_PCT'] = (all_games['FGM'] + 0.5 * all_games['FG3M']) / all_games['FGA']

# 2. Turnover Rate (Turnovers per 100 possessions)
# Estimate Possessions: FGA + 0.44*FTA - OREB + TOV
all_games['POSS_EST'] = all_games['FGA'] + 0.44 * all_games['FTA'] - all_games['OREB'] + all_games['TOV']
all_games['TOV_RATE'] = 100 * (all_games['TOV'] / all_games['POSS_EST'])

# 3. Offensive Rebounding (Raw proxy since we lack opp stats in this view)
all_games['OREB_RATE'] = all_games['OREB'] / all_games['POSS_EST'] # Proxy

# 4. Free Throw Rate (Ability to draw fouls)
all_games['FTA_RATE'] = all_games['FTA'] / all_games['FGA']

# 5. Pace (Speed of play)
all_games['PACE'] = 48 * (all_games['POSS_EST'] / (all_games['MIN'].astype(float) / 5))

# Target Variable
all_games['TARGET'] = all_games['WL'].apply(lambda x: 1 if x == 'W' else 0)

print("✅ Advanced Stats Calculated (Four Factors).")

# ==========================================
# 3. MERGE API STATS WITH ODDS
# ==========================================
name_map = {
    'LA Clippers': 'Los Angeles Clippers',
    'L.A. Clippers': 'Los Angeles Clippers',
    'L.A. Lakers': 'Los Angeles Lakers'
}
all_games['TEAM_NAME'] = all_games['TEAM_NAME'].replace(name_map)

# Create Odds Lookup Table
odds_lookup = pd.concat([
    train_odds[['Date', 'Home Team', 'Home_Prob']].rename(columns={'Date':'GAME_DATE', 'Home Team':'TEAM_NAME', 'Home_Prob':'ODDS_PROB'}),
    train_odds[['Date', 'Away Team', 'Away_Prob']].rename(columns={'Date':'GAME_DATE', 'Away Team':'TEAM_NAME', 'Away_Prob':'ODDS_PROB'})
])

# Merge
merged_data = pd.merge(all_games, odds_lookup, on=['GAME_DATE', 'TEAM_NAME'], how='left')
merged_data['ODDS_PROB'] = merged_data['ODDS_PROB'].fillna(0.5)

# Define the V2 Feature Set
features = [
    'ODDS_PROB',    # Market Sentiment (Crucial)
    'EFG_PCT',      # Shooting Efficiency
    'TOV_RATE',     # Ball Security
    'OREB_RATE',    # Rebounding
    'FTA_RATE',     # Aggression
    'PACE',         # Tempo
    'PLUS_MINUS'    # Overall Dominance
]

# ==========================================
# 4. TRAIN LSTM MODEL (V2 Architecture)
# ==========================================
print("\n--- STEP 3: Training Neural Network ---")

# Scale Data
scaler = MinMaxScaler()
merged_data[features] = scaler.fit_transform(merged_data[features])

def create_sequences(data):
    X, y = [], []
    for team in data['TEAM_ID'].unique():
        team_df = data[data['TEAM_ID'] == team].sort_values('GAME_DATE')
        if len(team_df) < SEQ_LENGTH + 1: continue

        vals = team_df[features].values
        targets = team_df['TARGET'].values

        for i in range(len(team_df) - SEQ_LENGTH):
            X.append(vals[i:i+SEQ_LENGTH])
            y.append(targets[i+SEQ_LENGTH])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(merged_data)

# Build Model (Optimized for current season size)
model = Sequential([
    Bidirectional(LSTM(32, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
                  input_shape=(SEQ_LENGTH, len(features))),
    Dropout(0.4),
    Bidirectional(LSTM(16, kernel_regularizer=regularizers.l2(0.01))),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=40, batch_size=8, verbose=0) # Silent training
print(f"✅ Model Trained on {len(X_train)} historical sequences.")





--- STEP 1: Loading Betting Data (nba_odds_full.csv) ---
✅ Loaded 288 past games for training.
✅ Found 5 games to predict tonight.

--- STEP 2: Fetching 2025-26 Game Stats ---
✅ Advanced Stats Calculated (Four Factors).

--- STEP 3: Training Neural Network ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model Trained on 503 historical sequences.


In [ ]:
# ==========================================
# 5. PREDICT TONIGHT'S GAMES
# ==========================================
print("\n--- 🏀 TONIGHT'S PREDICTIONS (V2 Model) ---")
results = []

for _, row in predict_odds.iterrows():
    home, away = row['Home Team'], row['Away Team']

    home_data = merged_data[merged_data['TEAM_NAME'] == home].sort_values('GAME_DATE').tail(SEQ_LENGTH)
    away_data = merged_data[merged_data['TEAM_NAME'] == away].sort_values('GAME_DATE').tail(SEQ_LENGTH)

    if len(home_data) < SEQ_LENGTH or len(away_data) < SEQ_LENGTH:
        results.append({'Matchup': f"{home} vs {away}", 'Signal': "Not enough data"})
        continue

    # Inject Live Odds
    home_input = home_data[features].values.copy()
    away_input = away_data[features].values.copy()
    home_input[-1, 0] = row['Home_Prob'] # Index 0 is ODDS_PROB
    away_input[-1, 0] = row['Away_Prob']

    # Predict
    p_home = model.predict(home_input.reshape(1, SEQ_LENGTH, len(features)), verbose=0)[0][0]

    edge = p_home - row['Home_Prob']

    if edge > 0.10: signal = f"🔥 BET {home}"
    elif edge > 0.05: signal = f"✅ BET {home}"
    elif edge < -0.10: signal = f"🔥 BET {away}"
    elif edge < -0.05: signal = f"✅ BET {away}"
    else: signal = "-- No Bet --"

    results.append({
        'Matchup': f"{home} vs {away}",
        'Bookie%': f"{row['Home_Prob']:.2f}",
        'Model%': f"{p_home:.2f}",
        'Edge': f"{edge:+.2f}",
        'Signal': signal
    })

final_df = pd.DataFrame(results)
print(final_df.to_string(index=False))


--- 🏀 TONIGHT'S PREDICTIONS (V2 Model) ---
                                       Matchup Bookie% Model%  Edge                      Signal
          Washington Wizards vs Boston Celtics    0.22   0.36 +0.14    🔥 BET Washington Wizards
   Philadelphia 76ers vs Golden State Warriors    0.64   0.47 -0.17 🔥 BET Golden State Warriors
         Toronto Raptors vs Los Angeles Lakers    0.58   0.48 -0.10    🔥 BET Los Angeles Lakers
                    Brooklyn Nets vs Utah Jazz    0.37   0.41 +0.03                -- No Bet --
New Orleans Pelicans vs Minnesota Timberwolves    0.19   0.48 +0.30  🔥 BET New Orleans Pelicans


In [ ]:
# ==========================================
# 6. INJURY AUDIT (PLAYER IMPACT) - CORRECTED
# ==========================================
from nba_api.stats.static import teams

print("\n--- 🏥 INJURY AUDIT: KEY PLAYERS TO CHECK ---")
print("If these players are OUT, ignore the 'BET' signal above.\n")
time.sleep(1)

# 1. Calculate WAR (Wins Above Replacement)
# Note: We use the column 'TEAM_ABBREVIATION' instead of 'TEAM_NAME' later
p_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=SEASON).get_data_frames()[0]
p_stats['IMPACT'] = (
    p_stats['PTS'] * 1.0 +
    p_stats['REB'] * 1.2 +
    p_stats['AST'] * 1.5 +
    p_stats['STL'] * 2.0 +
    p_stats['BLK'] * 2.0 -
    p_stats['TOV'] * 1.5
) / p_stats['GP']

max_score = p_stats['IMPACT'].max()
p_stats['WAR'] = (p_stats['IMPACT'] / max_score) * 100

# 2. Build a Map: "Los Angeles Lakers" -> "LAL"
nba_teams = teams.get_teams()
team_map = {team['full_name']: team['abbreviation'] for team in nba_teams}

# Add custom mappings for any CSV mismatches
# (e.g. if your CSV says "LA Clippers" instead of "Los Angeles Clippers")
team_map['LA Clippers'] = 'LAC'
team_map['L.A. Clippers'] = 'LAC'
team_map['L.A. Lakers'] = 'LAL'

# 3. Filter for Teams Playing Tonight
teams_playing = pd.concat([predict_odds['Home Team'], predict_odds['Away Team']]).unique()

for team_name in teams_playing:
    # Get the abbreviation (e.g., "BOS" for "Boston Celtics")
    # If name isn't found, default to the name itself (fallback)
    team_abbr = team_map.get(team_name)

    if not team_abbr:
        print(f"⚠️ Could not find abbreviation for {team_name}. Skipping.")
        continue

    # Filter using 'TEAM_ABBREVIATION'
    roster = p_stats[p_stats['TEAM_ABBREVIATION'] == team_abbr].sort_values('WAR', ascending=False)

    if not roster.empty:
        print(f"[{team_name}]")
        # Show Top 3 Players
        for _, p in roster.head(3).iterrows():
            star = "🌟" if p['WAR'] > 85 else ""
            print(f"  {star}{p['PLAYER_NAME']}: {p['WAR']:.1f}")
        print("")


--- 🏥 INJURY AUDIT: KEY PLAYERS TO CHECK ---
If these players are OUT, ignore the 'BET' signal above.

[Washington Wizards]
  Alex Sarr: 61.5
  Kyshawn George: 47.8
  CJ McCollum: 45.3

[Philadelphia 76ers]
  Tyrese Maxey: 84.4
  Joel Embiid: 52.3
  VJ Edgecombe: 48.7

[Toronto Raptors]
  Scottie Barnes: 66.7
  Brandon Ingram: 54.3
  Immanuel Quickley: 52.6

[Brooklyn Nets]
  Michael Porter Jr.: 63.8
  Nic Claxton: 51.8
  Cam Thomas: 42.3

[New Orleans Pelicans]
  Zion Williamson: 57.7
  Trey Murphy III: 56.6
  Saddiq Bey: 42.5

[Boston Celtics]
  Jaylen Brown: 68.8
  Derrick White: 54.2
  Payton Pritchard: 48.7

[Golden State Warriors]
  Stephen Curry: 62.7
  Jimmy Butler III: 59.1
  Brandin Podziemski: 39.3

[Los Angeles Lakers]
  🌟Luka Dončić: 96.0
  Austin Reaves: 71.4
  LeBron James: 49.5

[Utah Jazz]
  Lauri Markkanen: 66.1
  Keyonte George: 58.6
  Walker Kessler: 56.1

[Minnesota Timberwolves]
  Anthony Edwards: 68.4
  Julius Randle: 65.6
  Rudy Gobert: 46.7



In [ ]:
import pandas as pd
import requests
import io

# 1. Define URL
url = "https://www.espn.com/nba/injuries"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

try:
    # 2. Fetch Content
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # 3. Parse HTML (Fixed FutureWarning using io.StringIO)
    html_content = io.StringIO(response.text)
    dfs = pd.read_html(html_content)

    print(f"\n--- 🏥 NBA INJURY REPORT ({len(dfs)} Teams Found) ---\n")

    # 4. Process Tables
    for df in dfs:
        # Basic validation: Empty tables or weird shapes are skipped
        if df.empty or len(df.columns) < 3:
            continue

        # FORCE RENAME columns based on position (0=Player, 1=Status, 2=Reason)
        # This fixes the "KeyError" because we stop caring what ESPN named them.
        df.columns.values[0] = 'Player'
        df.columns.values[1] = 'Status'
        df.columns.values[2] = 'Reason'

        # Optional: Filter out garbage rows (sometimes headers repeat)
        df = df[df['Player'] != 'NAME']

        # Print simple report
        # We only print if we successfully renamed
        if 'Player' in df.columns and 'Status' in df.columns:
            print(df[['Player', 'Status', 'Reason']].to_string(index=False))
            print("-" * 50)

except Exception as e:
    print(f"❌ Error: {e}")


--- 🏥 NBA INJURY REPORT (29 Teams Found) ---

            Player Status Reason
      Jacob Toppin      F  Dec 6
     Jalen Johnson      F  Dec 5
Kristaps Porzingis      C  Dec 5
      N'Faly Dante      C  Dec 6
        Trae Young      G  Dec 6
--------------------------------------------------
      Player Status Reason
Jaylen Brown      G  Dec 5
Jayson Tatum      F  Apr 1
--------------------------------------------------
            Player Status Reason
      Drake Powell      G  Dec 6
Michael Porter Jr.      F  Dec 6
        Cam Thomas      G Dec 18
 Haywood Highsmith      F Dec 18
--------------------------------------------------
          Player Status Reason
Ryan Kalkbrenner      C  Dec 5
  Brandon Miller      F  Dec 5
 Pat Connaughton      G Dec 12
     LaMelo Ball      G  Dec 5
   Collin Sexton      G  Dec 7
        Tre Mann      G  Dec 5
  Grant Williams      F  Dec 7
      Josh Green      G  Dec 7
--------------------------------------------------
       Player Status Reaso

In [ ]:
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats

print("Initializing Player Database...")

try:
    # 1. Try to load the saved file
    player_db = pd.read_csv('nba_player_ratings.csv')
    print("✅ Loaded player ratings from CSV.")

except FileNotFoundError:
    print("⚠️ CSV not found. Fetching fresh stats from NBA API (2025-26)...")

    # 2. Fallback: Fetch live if file is missing
    stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2025-26').get_data_frames()[0]

    # Re-calculate the WAR/Impact Score
    stats['IMPACT'] = (
        stats['PTS'] + stats['REB']*1.2 + stats['AST']*1.5 +
        stats['STL']*2 + stats['BLK']*2 - stats['TOV']*1.5
    ) / stats['GP']

    stats['WAR_RATING'] = (stats['IMPACT'] / stats['IMPACT'].max()) * 100

    # Create the dataframe the other script is looking for
    player_db = stats[['PLAYER_NAME', 'TEAM_ABBREVIATION', 'WAR_RATING']]

    print("✅ Player stats fetched and calculated.")

print(f"Ready! Database contains {len(player_db)} players.")
print("Example:", player_db.sort_values('WAR_RATING', ascending=False).iloc[0]['PLAYER_NAME'])

Initializing Player Database...
⚠️ CSV not found. Fetching fresh stats from NBA API (2025-26)...
✅ Player stats fetched and calculated.
Ready! Database contains 492 players.
Example: Nikola Jokić


In [ ]:
# ==========================================
# 6. PREDICT TONIGHT'S GAMES (WITH INJURY ADJUSTMENT)
# ==========================================
print("\n--- 🏀 TONIGHT'S PREDICTIONS (Dec 4th) ---")

# 1. DEFINE WHO IS OUT (Manual Input from Injury Report)
# Add the names of key players confirmed OUT today
OUT_PLAYERS = ['Stephen Curry', 'Jimmy Butler']

# Load the ratings database we created in Step 6
try:
    player_db = pd.read_csv('nba_player_ratings.csv')
    # Create a quick lookup dict: Name -> WAR_RATING
    war_lookup = dict(zip(player_db['PLAYER_NAME'], player_db['WAR_RATING']))
except:
    print("⚠️ Warning: 'nba_player_ratings.csv' not found. Skipping injury adjustments.")
    war_lookup = {}

results = []

for _, row in predict_odds.iterrows():
    home, away = row['Home Team'], row['Away Team']

    # --- Standard Data Prep (Same as before) ---
    home_data = merged_data[merged_data['TEAM_NAME'] == home].sort_values('GAME_DATE').tail(SEQ_LENGTH)
    away_data = merged_data[merged_data['TEAM_NAME'] == away].sort_values('GAME_DATE').tail(SEQ_LENGTH)

    if len(home_data) < SEQ_LENGTH or len(away_data) < SEQ_LENGTH:
        results.append({'Matchup': f"{home} vs {away}", 'Signal': "Not enough data"})
        continue

    home_input = home_data[features].values.copy()
    away_input = away_data[features].values.copy()
    home_input[-1, 0] = row['Home_Prob']
    away_input[-1, 0] = row['Away_Prob']

    # --- Get Base Prediction ---
    p_home = model.predict(home_input.reshape(1, SEQ_LENGTH, len(features)), verbose=0)[0][0]

    # --- APPLY INJURY PENALTY ---
    # Formula: If a player is OUT, subtract (WAR_RATING * 0.20) from Win Prob
    # Example: Curry (WAR 98) -> Subtract 19.6% from Warriors' chance

    home_penalty = 0
    away_penalty = 0

    for player_name in OUT_PLAYERS:
        # Check if this player belongs to Home or Away team
        # We check the player_db to find their team
        player_row = player_db[player_db['PLAYER_NAME'] == player_name]
        if not player_row.empty:
            team_abbr = player_row.iloc[0]['TEAM_ABBREVIATION']
            rating = player_row.iloc[0]['WAR_RATING']

            # We need to map Abbr (GSW) back to Name (Golden State Warriors) to check match
            # Simple check: Is the team name in the current matchup?
            # (You might need a more robust Abbr->Name map if this misses, but this works for most)

            # Apply Penalty
            impact = (rating / 100) * 0.20 # Max penalty is 20% for a 100 WAR player

            if team_abbr in name_map.get(home, home) or home.startswith(team_abbr): # Loose match
                 # Wait, cleaner way: Check if player's team matches Home or Away Name
                 # Let's assume you know who plays for whom for now, or build a map.
                 # SIMPLE VERSION:
                 pass

    # Let's simplify the lookup logic for you:
    # We loop through the OUT_PLAYERS. If that player is on the Home Team, reduce Home Prob.

    notes = []

    for player in OUT_PLAYERS:
        # Find which team the player is on
        p_data = player_db[player_db['PLAYER_NAME'] == player]
        if p_data.empty: continue

        p_team_abbr = p_data.iloc[0]['TEAM_ABBREVIATION']
        p_war = p_data.iloc[0]['WAR_RATING']
        penalty = (p_war / 100) * 0.25 # 25% Impact for MVPs

        # Check if that team is in this matchup
        # Note: This requires mapping "GSW" to "Golden State Warriors"
        # For now, let's assume standard mapping or fuzzy string check

        # 1. Is Player on Home Team?
        # We cheat slightly by checking if the Abbreviation is "inside" the full name logic
        # OR use the nba_api team map we made earlier.
        # Let's just check if the Full Team Name contains the player's team logic.
        # (In a robust app, we'd use IDs).

        # Basic Check:
        # This relies on you getting the team name right in the csv, but works for this example
        # If we find the player's team is playing, we apply the penalty.

        # Let's define the penalty direction
        if p_team_abbr == 'GSW' and 'Warriors' in home:
            p_home -= penalty
            notes.append(f"{player} OUT (-{penalty:.0%})")
        elif p_team_abbr == 'GSW' and 'Warriors' in away:
            p_home += penalty # If Away is weaker, Home is stronger
            notes.append(f"{player} OUT (+{penalty:.0%})")

        # Add logic for other teams as needed or build a full map
        # For Butler (MIA):
        if p_team_abbr == 'MIA' and 'Heat' in home:
            p_home -= penalty
            notes.append(f"{player} OUT (-{penalty:.0%})")
        elif p_team_abbr == 'MIA' and 'Heat' in away:
            p_home += penalty
            notes.append(f"{player} OUT (+{penalty:.0%})")

    # --- Final Decision ---
    edge = p_home - row['Home_Prob']

    if edge > 0.10: signal = f"🔥 BET {home}"
    elif edge > 0.05: signal = f"✅ BET {home}"
    elif edge < -0.10: signal = f"🔥 BET {away}"
    elif edge < -0.05: signal = f"✅ BET {away}"
    else: signal = "-- No Bet --"

    results.append({
        'Matchup': f"{home} vs {away}",
        'Bookie%': f"{row['Home_Prob']:.2f}",
        'Adj_Model%': f"{p_home:.2f}", # Adjusted Probability
        'Edge': f"{edge:+.2f}",
        'Signal': signal,
        'Notes': ", ".join(notes)
    })

final_df = pd.DataFrame(results)
print(final_df.to_string(index=False))


--- 🏀 TONIGHT'S PREDICTIONS (Dec 4th) ---
⚠️ Warning: 'nba_player_ratings.csv' not found. Skipping injury adjustments.
                                       Matchup Bookie% Adj_Model%  Edge                     Signal                    Notes
          Washington Wizards vs Boston Celtics    0.22       0.36 +0.14   🔥 BET Washington Wizards                         
   Philadelphia 76ers vs Golden State Warriors    0.64       0.62 -0.01               -- No Bet -- Stephen Curry OUT (+16%)
         Toronto Raptors vs Los Angeles Lakers    0.58       0.48 -0.10   🔥 BET Los Angeles Lakers                         
                    Brooklyn Nets vs Utah Jazz    0.37       0.41 +0.03               -- No Bet --                         
New Orleans Pelicans vs Minnesota Timberwolves    0.19       0.48 +0.30 🔥 BET New Orleans Pelicans                         


In [ ]:
import pandas as pd
import requests
import io

print("--- 🏥 NBA INJURY REPORT (V2) ---")

url = "https://www.espn.com/nba/injuries"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Parse tables
    dfs = pd.read_html(io.StringIO(response.text))

    consolidated_injuries = []

    for i, df in enumerate(dfs):
        # Convert entire dataframe to string to search for headers
        # We look for the row that contains "NAME" or "PLAYER"

        # Strategy: Iterate through rows to find the header row
        header_row_idx = -1
        for idx, row in df.iterrows():
            row_str = row.astype(str).str.upper().tolist()
            if "NAME" in row_str or "PLAYER" in row_str:
                header_row_idx = idx
                break

        if header_row_idx != -1:
            # Set the header
            df.columns = df.iloc[header_row_idx]
            df = df.drop(df.index[:header_row_idx+1]).reset_index(drop=True)

        # Normalize columns
        # We look for columns containing specific keywords
        cols = df.columns.astype(str).str.upper()

        name_col = next((c for c in df.columns if "NAME" in str(c).upper() or "PLAYER" in str(c).upper()), None)
        status_col = next((c for c in df.columns if "STATUS" in str(c).upper()), None)
        date_col = next((c for c in df.columns if "DATE" in str(c).upper() or "RETURN" in str(c).upper()), None)

        if name_col and status_col:
            # Clean data
            subset = df[[name_col, status_col]].copy()
            subset.columns = ['Player', 'Status']

            # Add to master list
            for _, row in subset.iterrows():
                consolidated_injuries.append({
                    'Player': row['Player'],
                    'Status': row['Status']
                })

    # Create final dataframe
    injury_df = pd.DataFrame(consolidated_injuries)

    # Filter for "Out" or "Day-To-Day"
    if not injury_df.empty:
        print(f"Found {len(injury_df)} injured players.")
        print("\nSample of Data (First 10):")
        print(injury_df.head(10).to_string(index=False))

        # Save to CSV for the main script to use
        injury_df.to_csv('nba_injuries_today.csv', index=False)
        print("\n✅ Saved to 'nba_injuries_today.csv'")

        # Define the OUT_PLAYERS list for your next step automatically
        # We strictly take players listed as "Out"
        out_players_list = injury_df[injury_df['Status'].str.contains('Out', case=False, na=False)]['Player'].tolist()
        print(f"\n🚨 PLAYERS CONFIRMED OUT: {out_players_list}")

    else:
        print("⚠️ No injury data parsed. ESPN structure might have changed drastically.")

except Exception as e:
    print(f"❌ Error: {e}")

--- 🏥 NBA INJURY REPORT (V2) ---
Found 124 injured players.

Sample of Data (First 10):
            Player     Status
      Jacob Toppin        Out
     Jalen Johnson Day-To-Day
Kristaps Porzingis        Out
      N'Faly Dante Day-To-Day
        Trae Young        Out
      Jaylen Brown        Out
      Jayson Tatum        Out
      Drake Powell        Out
Michael Porter Jr.        Out
        Cam Thomas        Out

✅ Saved to 'nba_injuries_today.csv'

🚨 PLAYERS CONFIRMED OUT: ['Jacob Toppin', 'Kristaps Porzingis', 'Trae Young', 'Jaylen Brown', 'Jayson Tatum', 'Drake Powell', 'Michael Porter Jr.', 'Cam Thomas', 'Haywood Highsmith', 'Pat Connaughton', 'Collin Sexton', 'Grant Williams', 'Josh Green', 'Kevin Huerter', 'Isaac Okoro', 'Zach Collins', 'Jalen Smith', 'Noa Essengue', 'Coby White', 'Sam Merrill', 'Darius Garland', 'Max Strus', 'Jarrett Allen', 'Larry Nance Jr.', 'Dereck Lively II', 'P.J. Washington', 'Dante Exum', 'Kyrie Irving', 'Julian Strawther', 'Aaron Gordon', 'Christian Br

In [ ]:
# ==========================================
# 6. PREDICT WITH LIVE INJURY ADJUSTMENTS
# ==========================================
import pandas as pd
import numpy as np

print("\n--- 🏀 TONIGHT'S PREDICTIONS (FINAL) ---")

# 1. LOAD LIVE INJURIES
try:
    injuries = pd.read_csv('nba_injuries_today.csv')
    # Filter for players confirmed 'Out'
    # We convert to list for easy checking
    OUT_PLAYERS = injuries[injuries['Status'].isin(['Out'])]['Player'].tolist()
    print(f"⚠️ Loaded {len(OUT_PLAYERS)} inactive players for adjustment logic.")
except FileNotFoundError:
    print("❌ 'nba_injuries_today.csv' not found. Using manual list.")
    OUT_PLAYERS = ['Stephen Curry'] # Fallback

# 2. LOAD PLAYER RATINGS (WAR)
# We need this to know HOW important the missing player is
try:
    player_db = pd.read_csv('nba_player_ratings.csv')
except:
    # If missing, calculate on the fly (Safety net)
    from nba_api.stats.endpoints import leaguedashplayerstats
    stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2025-26').get_data_frames()[0]
    stats['IMPACT'] = (stats['PTS'] + stats['REB']*1.2 + stats['AST']*1.5 + stats['STL']*2 + stats['BLK']*2 - stats['TOV']*1.5) / stats['GP']
    stats['WAR_RATING'] = (stats['IMPACT'] / stats['IMPACT'].max()) * 100
    player_db = stats[['PLAYER_NAME', 'TEAM_ABBREVIATION', 'WAR_RATING']]

# 3. PREDICTION LOOP
results = []

for _, row in predict_odds.iterrows():
    home, away = row['Home Team'], row['Away Team']

    # --- Data Prep (Same as before) ---
    home_data = merged_data[merged_data['TEAM_NAME'] == home].sort_values('GAME_DATE').tail(SEQ_LENGTH)
    away_data = merged_data[merged_data['TEAM_NAME'] == away].sort_values('GAME_DATE').tail(SEQ_LENGTH)

    if len(home_data) < SEQ_LENGTH: continue

    home_input = home_data[features].values.copy()
    away_input = away_data[features].values.copy()
    home_input[-1, 4] = row['Home_Prob'] # Inject Odds
    away_input[-1, 4] = row['Away_Prob']

    # --- Base Prediction ---
    p_home = model.predict(home_input.reshape(1, SEQ_LENGTH, len(features)), verbose=0)[0][0]

    # --- INJURY PENALTY LOGIC ---
    current_notes = []

    # Check every "Out" player to see if they are on Home or Away team
    for player in OUT_PLAYERS:
        # Find player in our ratings DB
        p_row = player_db[player_db['PLAYER_NAME'] == player]

        if not p_row.empty:
            p_war = p_row.iloc[0]['WAR_RATING']
            p_team = p_row.iloc[0]['TEAM_ABBREVIATION']

            # Only apply penalty if WAR is significant (>60)
            if p_war > 60:
                # Calculate Impact (Max 20% penalty for MVP level)
                penalty = (p_war / 100) * 0.20

                # Check match (Simple string match for Team Name vs Abbr)
                # e.g. does "Boston Celtics" contain "BOS"? No, so we need the map.
                # We use the `team_map` we built earlier for abbreviation lookup

                # Get abbreviations for current matchup
                # (You might need to ensure team_map is available from Step 6 of previous chat)
                # Quick fallback map if needed:
                home_abbr = team_map.get(home, home[:3].upper())
                away_abbr = team_map.get(away, away[:3].upper())

                if p_team == home_abbr:
                    p_home -= penalty
                    current_notes.append(f"{player} OUT (-{penalty:.0%})")
                elif p_team == away_abbr:
                    p_home += penalty # Away player out helps Home team
                    current_notes.append(f"{player} OUT (+{penalty:.0%})")

    # --- Final Decision ---
    edge = p_home - row['Home_Prob']

    if edge > 0.10: signal = f"🔥 BET {home}"
    elif edge > 0.05: signal = f"✅ BET {home}"
    elif edge < -0.10: signal = f"🔥 BET {away}"
    elif edge < -0.05: signal = f"✅ BET {away}"
    else: signal = "-- No Bet --"

    results.append({
        'Matchup': f"{home} vs {away}",
        'Bookie%': f"{row['Home_Prob']:.2f}",
        'Adj_Model%': f"{p_home:.2f}",
        'Edge': f"{edge:+.2f}",
        'Signal': signal,
        'Notes': "; ".join(current_notes)
    })

# Output
final_df = pd.DataFrame(results)
print(final_df.to_string(index=False))


--- 🏀 TONIGHT'S PREDICTIONS (FINAL) ---
⚠️ Loaded 98 inactive players for adjustment logic.
                                       Matchup Bookie% Adj_Model%  Edge                     Signal                                         Notes
          Washington Wizards vs Boston Celtics    0.22       0.36 +0.14   🔥 BET Washington Wizards Jaylen Brown OUT (+14%); Alex Sarr OUT (-12%)
   Philadelphia 76ers vs Golden State Warriors    0.64       0.62 -0.01               -- No Bet --                      Stephen Curry OUT (+13%)
         Toronto Raptors vs Los Angeles Lakers    0.58       0.49 -0.09   ✅ BET Los Angeles Lakers                                              
                    Brooklyn Nets vs Utah Jazz    0.37       0.27 -0.10            🔥 BET Utah Jazz                 Michael Porter Jr. OUT (-13%)
New Orleans Pelicans vs Minnesota Timberwolves    0.19       0.47 +0.28 🔥 BET New Orleans Pelicans                                              
